In [54]:
import time
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import emoji
import Preprocessing as preproc
import nltk
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wojci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wojci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wojci\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\wojci\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [69]:
restaurants_raw = pd.read_csv(r"data_hyderabad/105_restaurants.csv")
reviews_raw = pd.read_csv(r"data_hyderabad/10k_reviews.csv")

restaurants_raw.head(5)

,Name,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Paradise,https://www.zomato.com/hyderabad/paradise-gach...,800,Hyderabad's Hottest,"Biryani, North Indian, Chinese",11 AM to 11 PM
2,Flechazo,https://www.zomato.com/hyderabad/flechazo-gach...,"1,300","Great Buffets, Hyderabad's Hottest","Asian, Mediterranean, North Indian, Desserts","11:30 AM to 4:30 PM, 6:30 PM to 11 PM"
3,Shah Ghouse Hotel & Restaurant,https://www.zomato.com/hyderabad/shah-ghouse-h...,800,Late Night Restaurants,"Biryani, North Indian, Chinese, Seafood, Bever...",12 Noon to 2 AM
4,Over The Moon Brew Company,https://www.zomato.com/hyderabad/over-the-moon...,"1,200","Best Bars & Pubs, Food Hygiene Rated Restauran...","Asian, Continental, North Indian, Chinese, Med...","12noon to 11pm (Mon, Tue, Wed, Thu, Sun), 12no..."


# Review Preprocessing

In [71]:
len(reviews_raw['Restaurant'].unique())
missing_restaurants = set(restaurants_raw['Name']) - set(reviews_raw['Restaurant'])
missing_restaurants

{'Angaara Counts 3',
 'IndiBlaze',
 'Republic Of Noodles - Lemon Tree Hotel',
 'Sweet Basket',
 'Wich Please'}

In [56]:
reviews_data = reviews_raw[reviews_raw["Rating"].notna() & reviews_raw["Review"].notna()]
reviews_data.isna().sum()

Restaurant    0
Reviewer      0
Review        0
Rating        0
Metadata      0
Time          0
Pictures      0
dtype: int64

In [57]:
# split UPPERCASE WORDS 
def splitting_words_process(word):
    # only upper case letters
    if word.isupper():
        return word
    
    # more than one upper case letter inside
    elif re.search(r'[A-Z][a-z]*[A-Z]', word):
        split_word = re.findall(r'[A-Z][a-z]*', word)
        return ' '.join(split_word)
    
    # <2 upper case letters
    else:
        return word

reviews_data['Review'] = reviews_data['Review'].apply(lambda x: ' '.join([splitting_words_process(word) for word in x.split()]))

In [58]:
# Function to replace 'gud', 'goo', 'gd' with the appropriate 'good'
def replace_gud_with_good(text):
    if isinstance(text, str):
        # Define the regex pattern to match 'gud', 'goo', 'gd' in various capitalizations
        pattern = re.compile(r'\b([Gg][Uu][Dd]|[Gg][Oo][Oo]|[Gg][Dd])\b')

        # Replacement function to check the case of the first letter
        def replacement(match):
            word = match.group()
            # Check if the first letter is uppercase, then return 'Good', else 'good'
            if word[0].isupper():
                return 'Good'
            else:
                return 'good'
        
        # Use re.sub to apply the replacement function
        return pattern.sub(replacement, text)
    
    return text

# Apply the function to the 'Review' column to replace the variants of 'good'
reviews_data['Review'] = reviews_data['Review'].apply(replace_gud_with_good)

In [59]:
# Function to replace 'kk', 'Oke', 'k', 'Ok' with 'ok'
def replace_to_ok(text):
    if isinstance(text, str):
        # Define the regex pattern to match the variants of 'ok'
        pattern = re.compile(r'\b(k|kk|Ok|Oke)\b', re.IGNORECASE)

        # Replacement function to return 'ok' for all matched words
        def replacement(match):
            return 'ok'
        
        # Use re.sub to apply the replacement function
        return pattern.sub(replacement, text)
    
    return text

# Apply the function to the 'Review' column to replace the variants of 'ok'
reviews_data['Review'] = reviews_data['Review'].apply(replace_to_ok)

In [60]:
# add space after ! | " | # | $ | % | & | ( | ) | * | + | , | . | : | ; followed immediately by a word
def add_space_after_punctuation(df):

    df['Review'] = df['Review'].apply(lambda text: re.sub(r'([\u0021-\u0026\u0028-\u002C\u002E\u003A-\u003F]+(?=\w))', r'\1 ', text) if isinstance(text, str) else text)
    return df

# Example usage:
reviews_data = add_space_after_punctuation(reviews_data)

In [61]:
# remove gibberish words like "ggggggggggd", "eshjdgue"
def remove_gibberish(text):
    cleaned_text = re.sub(r'\b\w{15,}\b', '', text)  # removes 15+ words
    cleaned_text = re.sub(r'\b\w*(\w)\1{2,}\w*\b', '', cleaned_text)  # removes words that contain 3+ repeating letters

    return cleaned_text

reviews_data['Review'] = reviews_data['Review'].apply(remove_gibberish)

In [62]:
# replace numbers with blank string
reviews_data['Review'] = reviews_data['Review'].replace(r'\d+(\.\d+)?', '', regex=True)

In [ ]:
preproc = reviews_data['Review'].apply(lambda x: preproc.main_pipeline(
    x, 
    print_output=False, 
    no_stopwords=False,
    custom_stopwords=[],
    convert_diacritics=True, 
    no_punctuation=False,
    remove_contractions = True,
    lowercase=False,
    lemmatized=False,
    list_pos=["n","v","a","r","s"],
    stemmed=False, 
    pos_tags_list='pos_tuples',
    tokenized_output=False
))

In [64]:
# remove stopwords
def remove_stopwords(tokens):
    punctuation_pattern = "[\u0021-\u0026\u0028-\u002C\u002E-\u002F\u003A-\u003F\u005B-\u005F\u2010-\u2028\ufeff`]+"
    stopwords = set(nltk.corpus.stopwords.words("english"))

    return [(token.lower(), pos) for token, pos in tokens if token.lower() not in stopwords and not re.match(punctuation_pattern, token)]

reviews_data['Review_Preprocessed'] = preproc.apply(remove_stopwords)

In [65]:
data_joined = pd.merge(reviews_data, restaurants_raw, left_on='Restaurant',right_on='Name', how='left')
data_joined.head(1)

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,Review_Preprocessed,Name,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,"[(ambience, NN), (good, JJ), (food, NN), (quit...",Beyond Flavours,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"


In [66]:
len(data_joined['Restaurant'].unique())

100

In [15]:
data_joined = data_joined[['Restaurant', 'Review', 'Review_Preprocessed', 'Cuisines']]
data_joined.head(3)

,Restaurant,Review,Review_Preprocessed,Cuisines
0,Beyond Flavours,"The ambience was good, food was quite good . h...","[(ambience, NN), (good, JJ), (food, NN), (quit...","Chinese, Continental, Kebab, European, South I..."
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,"[(ambience, NN), (good, JJ), (pleasant, JJ), (...","Chinese, Continental, Kebab, European, South I..."
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[(must, MD), (try, VB), (great, JJ), (food, NN...","Chinese, Continental, Kebab, European, South I..."


In [16]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer_roberta = AutoTokenizer.from_pretrained("Dizex/InstaFoodRoBERTa-NER")
model_roberta = AutoModelForTokenClassification.from_pretrained("Dizex/InstaFoodRoBERTa-NER")

In [17]:
pipe = pipeline("ner", model=model_roberta, tokenizer=tokenizer_roberta)

In [18]:
from tqdm import tqdm

ner_entity_results = pipe(list(data_joined['Review']), aggregation_strategy="simple")

KeyboardInterrupt: 

In [ ]:
def convert_entities_to_list(df: pd.DataFrame, entities: list[list[dict]]) -> pd.DataFrame:
    def extract_entities(text, entity_list):
        ents = []
        for ent in entity_list:
            e = {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
            if ents and (-1 <= ent["start"] - ents[-1]["end"] <= 1) and (ents[-1]["label"] == e["label"]):
                ents[-1]["end"] = e["end"]
                continue
            ents.append(e)
        return [text[e["start"]:e["end"]] for e in ents]

    df['meals'] = [extract_entities(text, entity_list) for text, entity_list in zip(df['Review'], entities)]
    return df

In [ ]:
data_joined = convert_entities_to_list(data_joined, ner_entity_results)

,Restaurant,Review,Review_Preprocessed,Cuisines,meals
0,Beyond Flavours,"The ambience was good, food was quite good . h...","[(ambience, NN), (good, JJ), (food, NN), (quit...","Chinese, Continental, Kebab, European, South I...",[]
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,"[(ambience, NN), (good, JJ), (pleasant, JJ), (...","Chinese, Continental, Kebab, European, South I...",[]
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[(must, MD), (try, VB), (great, JJ), (food, NN...","Chinese, Continental, Kebab, European, South I...",[Penne Alfredo Pasta]
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"[(soumen, NNP), (das, NNS), (arun, NNP), (grea...","Chinese, Continental, Kebab, European, South I...",[]
4,Beyond Flavours,Food is good. we ordered Kodi drumsticks and b...,"[(food, NN), (good, JJ), (ordered, VBD), (kodi...","Chinese, Continental, Kebab, European, South I...","[Kodi drumsticks, basket mutton biryani]"
...,...,...,...,...,...
95,Beyond Flavours,Was there for office lunch outing. Rating woul...,"[(office, NN), (lunch, NN), (outing, VBG), (ra...","Chinese, Continental, Kebab, European, South I...",[]
96,Beyond Flavours,"I really enjoyed the follows.... The entrance,...","[(really, RB), (enjoyed, VBD), (entrance, NN),...","Chinese, Continental, Kebab, European, South I...",[]
97,Beyond Flavours,I came first time in this restaurant. The entr...,"[(came, VBD), (first, JJ), (time, NN), (restau...","Chinese, Continental, Kebab, European, South I...",[]
98,Beyond Flavours,Pathetic and horrible experience Ambience and ...,"[(pathetic, JJ), (horrible, JJ), (experience, ...","Chinese, Continental, Kebab, European, South I...",[]


# Find out all the possible labels

In [145]:
from sklearn.preprocessing import MultiLabelBinarizer

def split_list_into_onehot_labels(dataframe, column_name):
    """
    Splits a list of comma-separated values in a specified column of a DataFrame into one-hot encoded labels.
    Args:
        dataframe (pd.DataFrame): The input DataFrame containing the data.
        column_name (str): The name of the column containing comma-separated values to be one-hot encoded.
    Returns:
        pd.DataFrame: The DataFrame with the original column split into a single column containing one-hot encoded vectors.

    """
    mlb = MultiLabelBinarizer()
    dataframe[column_name] = dataframe[column_name].apply(lambda x: x.split(", "))

    cuisine_encoded = mlb.fit_transform(dataframe[column_name])

    dataframe['Cuisine_Vector'] = list(cuisine_encoded)
    return dataframe

In [182]:
import pandas as pd
data_joined = pd.read_csv(r"data_hyderabad/data_preprocessed_classification.csv")

#data_joined['meals'] = data_joined['meals'].apply(lambda x: [meal.lower() for meal in eval(x)])


data_joined = split_list_into_onehot_labels(data_joined, 'Cuisines')
data_joined.head(3)

,Restaurant,Review,Review_Preprocessed,Cuisines,meals,Cuisine_Vector
0,Beyond Flavours,"The ambience was good, food was quite good . h...","[('ambience', 'NN'), ('good', 'JJ'), ('food', ...","[Chinese, Continental, Kebab, European, South ...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,"[('ambience', 'NN'), ('good', 'JJ'), ('pleasan...","[Chinese, Continental, Kebab, European, South ...",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[('must', 'MD'), ('try', 'VB'), ('great', 'JJ'...","[Chinese, Continental, Kebab, European, South ...",['Penne Alfredo Pasta'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."


In [183]:
# Filter out rows where 'meals' column is empty
data_joined = data_joined[data_joined['meals'].apply(lambda x: x != "[]")]

# Convert 'Review_Preprocessed' from string to list of tuples
data_joined['Review_Preprocessed'] = data_joined['Review_Preprocessed'].apply(eval)

# Extract tokens from 'Review_Preprocessed' and store in a new column
data_joined['Review_Preprocessed_No_Pos'] = data_joined['Review_Preprocessed'].apply(lambda x: [token for token, pos in x])

# Reorder columns to place 'Review_Preprocessed_No_Pos' after 'Review_Preprocessed'
cols = list(data_joined.columns)
review_index = cols.index('Review_Preprocessed')
cols.insert(review_index + 1, cols.pop(cols.index('Review_Preprocessed_No_Pos')))
data_joined = data_joined[cols]

# Create a copy of the dataframe for further processing
data_preprocessed_many_rows = data_joined.copy()

# Group by 'Restaurant' and aggregate reviews and meals
data_preprocessed_groupedby_restaurant = data_preprocessed_many_rows.groupby('Restaurant').agg({
    'Review': lambda x: ', '.join(x),
    'Review_Preprocessed_No_Pos': lambda x: ', '.join([', '.join(tokens) for tokens in x]),
    'Review_Preprocessed': lambda x: ', '.join([', '.join([f"({token}, {pos})" for token, pos in tokens]) for tokens in x]),
    'meals': lambda x: ', '.join([meal for sublist in x for meal in eval(sublist)])
}).reset_index().rename(columns={"Review_Preprocessed": "Review_Preprocessed_Pos"})

data_preprocessed_groupedby_restaurant

,Restaurant,Review,Review_Preprocessed_No_Pos,Review_Preprocessed_Pos,meals
0,10 Downing Street,I've been to this place about two times and i ...,"'ve, place, two, times, really, liked, ambienc...","('ve, VBP), (place, NN), (two, CD), (times, NN...","lasagna, veg Platter, lasagna rolls, beers, ve..."
1,13 Dhaba,I didn't go and eat at the Dhaba. I had ordere...,"go, eat, dhaba, ordered, taste, amazing, te, i...","(go, VB), (eat, VB), (dhaba, NNP), (ordered, V...","lassi, Chole bhature, Lassi, chole bhature pan..."
2,"3B's - Buddies, Bar & Barbecue",Gobind Passionate in serving Polite in nature ...,"gobind, passionate, serving, polite, nature, s...","(gobind, NNP), (passionate, NNP), (serving, VB...","Polite, Pan ice cream, pan ice cream, pan ice ..."
3,AB's - Absolute Barbecues,Excellent service by nandan and rahmat and rip...,"excellent, service, nandan, rahmat, ripan, fee...","(excellent, JJ), (service, NN), (nandan, NN), ...","ripan, politley sarvice, fish, pankaj, cake, b..."
4,Absolute Sizzlers,Service was pathetic. Ordered a sizzler with l...,"service, pathetic, ordered, sizzler, lamb, tol...","(service, NNP), (pathetic, JJ), (ordered, VBD)...","ler, lamb, lamb, Noodles, rice, noodle, chilli..."
...,...,...,...,...,...
95,Urban Asia - Kitchen & Bar,This place is highly recommended. It is workin...,"place, highly, recommended, working, eat, indi...","(place, NN), (highly, RB), (recommended, JJ), ...","noodles, Sanghai Fried Rice, Fish, sauce, nood..."
96,Yum Yum Tree - The Arabian Food Court,It is at th floor of Act Boutique building tha...,"th, floor, act, boutique, building, entrance, ...","(th, JJ), (floor, NN), (act, NNP), (boutique, ...","mutton Haleem, Chicken Fahm Mandi, chicken hal..."
97,Zega - Sheraton Hyderabad Hotel,"My husband and I, visited Zega for their dimsu...","husband, visited, zega, dimsum, festival, disa...","(husband, NN), (visited, VBD), (zega, NNP), (d...","thukpa, spice, dimsums, chicken Gyoza, dimsums..."
98,Zing's Northeast Kitchen,After so many of goody goody excellent reviews...,"many, goody, goody, excellent, reviews, n, exc...","(many, JJ), (goody, NN), (goody, NN), (excelle...","chalega, Pork, beef, meat, meat, veg momo, veg..."


In [184]:
# Merge 'Cuisine_Vector' from 'data_joined' into 'data_preprocessed_groupedby_restaurant'
data_preprocessed_groupedby_restaurant = data_preprocessed_groupedby_restaurant.merge(
    data_joined[['Restaurant', 'Cuisine_Vector']],
    on='Restaurant',
    how='left'
)

# Display the updated DataFrame
data_preprocessed_groupedby_restaurant.drop_duplicates(subset=['Restaurant'], inplace=True)
data_preprocessed_groupedby_restaurant.reset_index(drop=True)

,Restaurant,Review,Review_Preprocessed_No_Pos,Review_Preprocessed_Pos,meals,Cuisine_Vector
0,10 Downing Street,I've been to this place about two times and i ...,"'ve, place, two, times, really, liked, ambienc...","('ve, VBP), (place, NN), (two, CD), (times, NN...","lasagna, veg Platter, lasagna rolls, beers, ve...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
1,13 Dhaba,I didn't go and eat at the Dhaba. I had ordere...,"go, eat, dhaba, ordered, taste, amazing, te, i...","(go, VB), (eat, VB), (dhaba, NNP), (ordered, V...","lassi, Chole bhature, Lassi, chole bhature pan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"3B's - Buddies, Bar & Barbecue",Gobind Passionate in serving Polite in nature ...,"gobind, passionate, serving, polite, nature, s...","(gobind, NNP), (passionate, NNP), (serving, VB...","Polite, Pan ice cream, pan ice cream, pan ice ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,AB's - Absolute Barbecues,Excellent service by nandan and rahmat and rip...,"excellent, service, nandan, rahmat, ripan, fee...","(excellent, JJ), (service, NN), (nandan, NN), ...","ripan, politley sarvice, fish, pankaj, cake, b...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,Absolute Sizzlers,Service was pathetic. Ordered a sizzler with l...,"service, pathetic, ordered, sizzler, lamb, tol...","(service, NNP), (pathetic, JJ), (ordered, VBD)...","ler, lamb, lamb, Noodles, rice, noodle, chilli...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
...,...,...,...,...,...,...
95,Urban Asia - Kitchen & Bar,This place is highly recommended. It is workin...,"place, highly, recommended, working, eat, indi...","(place, NN), (highly, RB), (recommended, JJ), ...","noodles, Sanghai Fried Rice, Fish, sauce, nood...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
96,Yum Yum Tree - The Arabian Food Court,It is at th floor of Act Boutique building tha...,"th, floor, act, boutique, building, entrance, ...","(th, JJ), (floor, NN), (act, NNP), (boutique, ...","mutton Haleem, Chicken Fahm Mandi, chicken hal...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
97,Zega - Sheraton Hyderabad Hotel,"My husband and I, visited Zega for their dimsu...","husband, visited, zega, dimsum, festival, disa...","(husband, NN), (visited, VBD), (zega, NNP), (d...","thukpa, spice, dimsums, chicken Gyoza, dimsums...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,Zing's Northeast Kitchen,After so many of goody goody excellent reviews...,"many, goody, goody, excellent, reviews, n, exc...","(many, JJ), (goody, NN), (goody, NN), (excelle...","chalega, Pork, beef, meat, meat, veg momo, veg...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [186]:
data_preprocessed_groupedby_restaurant['meals'] = data_preprocessed_groupedby_restaurant['meals'].apply(lambda x: x.split())
data_preproc_grouped = data_preprocessed_groupedby_restaurant.copy()

data_joined['meals'] = data_joined['meals'].apply(lambda x: [meal.lower() for meal in eval(x)])
data_preproc_review_per_row = data_joined.copy()

In [187]:
data_preproc_grouped

,Restaurant,Review,Review_Preprocessed_No_Pos,Review_Preprocessed_Pos,meals,Cuisine_Vector
0,10 Downing Street,I've been to this place about two times and i ...,"'ve, place, two, times, really, liked, ambienc...","('ve, VBP), (place, NN), (two, CD), (times, NN...","[lasagna,, veg, Platter,, lasagna, rolls,, bee...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
61,13 Dhaba,I didn't go and eat at the Dhaba. I had ordere...,"go, eat, dhaba, ordered, taste, amazing, te, i...","(go, VB), (eat, VB), (dhaba, NNP), (ordered, V...","[lassi,, Chole, bhature,, Lassi,, chole, bhatu...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
132,"3B's - Buddies, Bar & Barbecue",Gobind Passionate in serving Polite in nature ...,"gobind, passionate, serving, polite, nature, s...","(gobind, NNP), (passionate, NNP), (serving, VB...","[Polite,, Pan, ice, cream,, pan, ice, cream,, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
159,AB's - Absolute Barbecues,Excellent service by nandan and rahmat and rip...,"excellent, service, nandan, rahmat, ripan, fee...","(excellent, JJ), (service, NN), (nandan, NN), ...","[ripan,, politley, sarvice,, fish,, pankaj,, c...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
193,Absolute Sizzlers,Service was pathetic. Ordered a sizzler with l...,"service, pathetic, ordered, sizzler, lamb, tol...","(service, NNP), (pathetic, JJ), (ordered, VBD)...","[ler,, lamb,, lamb,, Noodles,, rice,, noodle,,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
...,...,...,...,...,...,...
5561,Urban Asia - Kitchen & Bar,This place is highly recommended. It is workin...,"place, highly, recommended, working, eat, indi...","(place, NN), (highly, RB), (recommended, JJ), ...","[noodles,, Sanghai, Fried, Rice,, Fish,, sauce...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
5627,Yum Yum Tree - The Arabian Food Court,It is at th floor of Act Boutique building tha...,"th, floor, act, boutique, building, entrance, ...","(th, JJ), (floor, NN), (act, NNP), (boutique, ...","[mutton, Haleem,, Chicken, Fahm, Mandi,, chick...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5700,Zega - Sheraton Hyderabad Hotel,"My husband and I, visited Zega for their dimsu...","husband, visited, zega, dimsum, festival, disa...","(husband, NN), (visited, VBD), (zega, NNP), (d...","[thukpa,, spice,, dimsums,, chicken, Gyoza,, d...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5753,Zing's Northeast Kitchen,After so many of goody goody excellent reviews...,"many, goody, goody, excellent, reviews, n, exc...","(many, JJ), (goody, NN), (goody, NN), (excelle...","[chalega,, Pork,, beef,, meat,, meat,, veg, mo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [188]:
data_preproc_review_per_row

,Restaurant,Review,Review_Preprocessed,Review_Preprocessed_No_Pos,Cuisines,meals,Cuisine_Vector
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[(must, MD), (try, VB), (great, JJ), (food, NN...","[must, try, great, food, great, ambience, thnx...","[Chinese, Continental, Kebab, European, South ...",[penne alfredo pasta],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
4,Beyond Flavours,Food is good. we ordered Kodi drumsticks and b...,"[(food, NN), (good, JJ), (ordered, VBD), (kodi...","[food, good, ordered, kodi, drumsticks, basket...","[Chinese, Continental, Kebab, European, South ...","[kodi drumsticks, basket mutton biryani]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
7,Beyond Flavours,Well after reading so many reviews finally vis...,"[(well, RB), (reading, VBG), (many, JJ), (revi...","[well, reading, many, reviews, finally, visite...","[Chinese, Continental, Kebab, European, South ...","[corn, tawa fish, basket biryani, biryani]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
9,Beyond Flavours,Came for the birthday treat of a close friend....,"[(came, NN), (birthday, JJ), (treat, NN), (clo...","[came, birthday, treat, close, friend, perfect...","[Chinese, Continental, Kebab, European, South ...",[chili honey lotus stem],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
12,Beyond Flavours,Food was very good. Soup was as expected. In s...,"[(food, NN), (good, JJ), (soup, NNP), (expecte...","[food, good, soup, expected, starters, ordered...","[Chinese, Continental, Kebab, European, South ...","[soup, honey chilli lotus]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
...,...,...,...,...,...,...,...
9949,Chinese Pavilion,Chinese Pavilion in Banjara H Ills was the fir...,"[(chinese, JJ), (pavilion, NNP), (banjara, NNP...","[chinese, pavilion, banjara, h, ills, first, c...","[Chinese, Seafood]","[noodles, fried rice, lamb, brownie, rice, lamb]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
9950,Chinese Pavilion,Madhumathi Mahajan Well to start with nice cou...,"[(madhumathi, NNP), (mahajan, NNP), (well, NNP...","[madhumathi, mahajan, well, start, nice, court...","[Chinese, Seafood]","[jade chicken soup, spinach, house salad, cabb...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
9951,Chinese Pavilion,This place has never disappointed us.. The foo...,"[(place, NN), (never, RB), (disappointed, VBN)...","[place, never, disappointed, us, food, courteo...","[Chinese, Seafood]","[rice, oil, fish, garlic noodles, chicken]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
9952,Chinese Pavilion,"Bad rating is mainly because of "" Chicken Bone...","[(bad, NNP), (rating, NN), (mainly, RB), (chic...","[bad, rating, mainly, chicken, bone, found, ve...","[Chinese, Seafood]","[chicken bone, hot and sour soup, sprouts, kim...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


# **Futher steps**

## Final preprocessing steps:
1. Create two datasets: 
2. 1st with 9k+ rows, so every review has a label vector
3. 2nd with 105 rows, so every restaurant has a joint review vector of food names


## Modeling
I see two possible approaches to solve this problem. 

1. We treat the dishes as categorical instances and build a model that takes counts
It will be a categorical to categorical model so Decision Tree or Naive Bayes should do

2. Encoding the words with (no tf-idf not appropriate for this appraoch coz no documents), Word2Vec and Glove and categorizing the labels based on those. Here we can think about adding/multiplying the vectors to get the overall or dot product and using this numerical representation for classification

In [212]:
from gensim.models import Word2Vec

# Train the Word2Vec model on the 'meals' column
reviews_skipgram_model = Word2Vec(
    sentences=data_preproc_review_per_row["Review_Preprocessed_No_Pos"].tolist(),
    vector_size=25,
    window=5,
    min_count=1,
    workers=4,
    sg=1  # Skip-gram model
)

#Generate vectors for each review
def get_review_vector(review):
    words = review
    vector = sum(reviews_skipgram_model.wv[word] for word in words if word in reviews_skipgram_model.wv)
    return vector / len(words) if words else [0] * 25

# Apply the function to each review and create a new column
data_preproc_review_per_row['Word2Vec_Vector'] = data_preproc_review_per_row['Review_Preprocessed_No_Pos'].apply(get_review_vector)

# Display the DataFrame with the new column
data_preproc_review_per_row.head()

,Restaurant,Review,Review_Preprocessed,Review_Preprocessed_No_Pos,Cuisines,meals,Cuisine_Vector,Word2Vec_Vector
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[(must, MD), (try, VB), (great, JJ), (food, NN...","[must, try, great, food, great, ambience, thnx...","[Chinese, Continental, Kebab, European, South ...",[penne alfredo pasta],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[0.09448282, 0.20344338, 0.1122126, -0.5233302..."
4,Beyond Flavours,Food is good. we ordered Kodi drumsticks and b...,"[(food, NN), (good, JJ), (ordered, VBD), (kodi...","[food, good, ordered, kodi, drumsticks, basket...","[Chinese, Continental, Kebab, European, South ...","[kodi drumsticks, basket mutton biryani]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[0.07203948, 0.27816552, -0.031543814, -0.4987..."
7,Beyond Flavours,Well after reading so many reviews finally vis...,"[(well, RB), (reading, VBG), (many, JJ), (revi...","[well, reading, many, reviews, finally, visite...","[Chinese, Continental, Kebab, European, South ...","[corn, tawa fish, basket biryani, biryani]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[0.050413016, 0.18470684, 0.061515264, -0.6186..."
9,Beyond Flavours,Came for the birthday treat of a close friend....,"[(came, NN), (birthday, JJ), (treat, NN), (clo...","[came, birthday, treat, close, friend, perfect...","[Chinese, Continental, Kebab, European, South ...",[chili honey lotus stem],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[0.004651741, 0.12694506, 0.08261496, -0.63544..."
12,Beyond Flavours,Food was very good. Soup was as expected. In s...,"[(food, NN), (good, JJ), (soup, NNP), (expecte...","[food, good, soup, expected, starters, ordered...","[Chinese, Continental, Kebab, European, South ...","[soup, honey chilli lotus]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[0.11184975, 0.23971973, 0.05759144, -0.590902..."


In [236]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm

def cooccurrence_matrix_sentence_generator(preproc_sentences):
    # Flatten the list of sentences to get all unique words
    unique_words = list(set(word for sentence in preproc_sentences for word in sentence))
    word_index = {word: idx for idx, word in enumerate(unique_words)}
    vocab_size = len(unique_words)

    # Initialize the co-occurrence matrix
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=int)

    # Compute co-occurrences
    for sentence in tqdm(preproc_sentences):
        word_indices = [word_index[word] for word in sentence if word in word_index]
        for i in range(len(word_indices)):
            co_matrix[word_indices[i], word_indices[i:]] += 1
            co_matrix[word_indices[i:], word_indices[i]] += 1

    # Create a DataFrame for better readability
    co_matrix_df = pd.DataFrame(co_matrix, index=unique_words, columns=unique_words)

    co_matrix_df = co_matrix_df.reindex(co_matrix_df.sum().sort_values(ascending=False).index, axis=1)
    co_matrix_df = co_matrix_df.reindex(co_matrix_df.sum().sort_values(ascending=False).index, axis=0)

    # Return the co-occurrence matrix
    return co_matrix_df

In [238]:
co_occr_matrix = cooccurrence_matrix_sentence_generator(data_preproc_grouped['meals'])

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:04<00:00, 21.32it/s]


In [240]:
co_occr_matrix

,chicken,"chicken,","biryani,",Chicken,"rice,",veg,"soup,","veg,","pizza,","curry,",...,"Sm,","nutral,","doubles,","uttappam,","WASTE,","ulavacharu,","Dominos,",Dominos,rayalsima,"domin,"
chicken,2326,2011,1556,1438,1305,1075,829,922,543,820,...,1,6,1,0,1,3,0,0,1,0
"chicken,",2011,2138,1452,1370,1292,1011,820,879,480,882,...,1,3,1,0,1,2,0,0,1,0
"biryani,",1556,1452,1852,1108,1104,775,528,648,136,709,...,1,10,0,0,0,7,0,0,4,0
Chicken,1438,1370,1108,1190,895,696,630,611,416,552,...,0,0,0,0,1,0,0,0,0,0
"rice,",1305,1292,1104,895,1208,668,668,559,252,688,...,0,1,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"ulavacharu,",3,2,7,0,1,2,0,0,0,0,...,0,1,0,0,0,2,0,0,1,0
"Dominos,",0,0,0,0,0,1,0,1,10,0,...,0,0,0,1,0,0,2,1,0,1
Dominos,0,0,0,0,0,1,0,1,9,0,...,0,0,0,1,0,0,1,2,0,1
rayalsima,1,1,4,0,1,2,0,0,0,0,...,0,1,0,0,0,1,0,0,2,0


In [239]:
from mittens import GloVe
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Train GloVe model on the co-occurrence matrix
glove_model = GloVe(n=25, max_iter=150, display_progress=10)  # 25 is the embedding dimension
embeddings = glove_model.fit(co_occr_matrix)

# Generate vectors for each review using GloVe embeddings
def get_review_vector_glove(review, embeddings, vocab, vector_size=25):
    words = review.split()
    vector = np.sum([embeddings[vocab[word]] for word in words if word in vocab], axis=0)
    return vector / len(words) if words else np.zeros(vector_size)

# Apply the function to each review and create a new column
data_preproc_review_per_row['Glove_Vector'] = data_preproc_review_per_row['Review_Preprocessed_No_Pos'].apply(lambda x: get_review_vector_glove(x, embeddings, vocab))

# Assuming 'cuisine_vector' is the target column
X = np.array(data_preproc_review_per_row['Glove_Vector'].tolist())
y = np.array(data_preproc_review_per_row['cuisine_vector'].tolist())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Predict the cuisine vector
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

AttributeError: 'Series' object has no attribute 'reshape'

In [210]:
X = data_preproc_review_per_row['Word2Vec_Vector'].to_list()
y = data_preproc_review_per_row['Cuisine_Vector'].to_list()

In [215]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

X_one_hot = data_preproc_review_per_row['meals'].copy()
X_one_hot = onehot.fit_transform(X).toarray()
X_one_hot.shape

(5904, 123326)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,classification_report
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#moc = MultiOutputClassifier(GaussianNB())                                                #f1_score: 0.34
moc = MultiOutputClassifier(LogisticRegression(solver='lbfgs', class_weight="balanced"))  #f1_score: 0.45 on reviews_rows / f1_score: 0.40
#moc = MultiOutputClassifier(RandomForestClassifier())                                    #f1_score: 0.34

moc.fit(X_train, y_train)
y_pred = moc.predict(X_test)

y_pred = moc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.32      0.28       113
           1       0.07      0.30      0.11        99
           2       0.19      0.15      0.17        40
           3       0.00      0.00      0.00       240
           4       0.00      0.00      0.00        31
           5       0.27      0.40      0.32       154
           6       0.11      0.22      0.15       108
           7       0.49      0.39      0.44       249
           8       0.14      0.48      0.22        96
           9       0.36      0.37      0.36       145
          10       0.50      0.57      0.53       680
          11       0.23      0.49      0.31       358
          12       0.57      0.33      0.41       280
          13       0.04      0.18      0.07        55
          14       0.29      0.18      0.22       240
          15       0.00      0.00      0.00        27
          16       0.00      0.00      0.00        21
          17       0.02    

c:\Users\wojci\anaconda3\envs\text-mining1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wojci\anaconda3\envs\text-mining1\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
